# **Entregable Semana 3**

# **📊 Nuevo Script de Análisis de Datos Meteorológicos**
Este script:

Lee archivos .csv y .json.

Extrae campos numéricos (temperatura, humedad, presión).

Calcula:

Media y desviación estándar de la temperatura.

Humedad promedio.

Presión mínima y máxima.

**Código principal**: Es necesario para generar los archivos que el script de análisis leera para recolectar los datos necesarios.

In [ ]:
import requests
import json
import csv
from datetime import datetime, timezone
import time

API_key = "f0f0c07c117721896152aeca0fbbe31a"

def obtener_clima_actual(ciudad):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={ciudad}&units=metric&appid={API_key}"
    res = requests.get(url)
    if res.status_code != 200:
        print("❌ Ciudad no encontrada o error en la petición.")
        return None
    data = res.json()
    temperatura = data["main"]["temp"]
    humedad = data["main"]["humidity"]
    presion = data["main"]["pressure"]
    return {
        "fecha": datetime.now().strftime("%Y-%m-%d"),
        "ciudad": ciudad,
        "temperatura": temperatura,
        "humedad": humedad,
        "presion": presion,
        "tipo": "actual"
    }

def obtener_pronostico(ciudad):
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={ciudad}&units=metric&appid={API_key}"
    res = requests.get(url)
    if res.status_code != 200:
        print("❌ Ciudad no encontrada o error en la petición.")
        return []
    data = res.json()
    pronosticos = []

    # Queremos solo 1 pronóstico por día a las 18:00 hrs
    fechas_procesadas = set()
    for item in data["list"]:
        dt_txt = item["dt_txt"]  # formato 'YYYY-MM-DD HH:MM:SS'
        fecha_str, hora_str = dt_txt.split()
        if hora_str == "18:00:00" and fecha_str not in fechas_procesadas:
            fechas_procesadas.add(fecha_str)
            pronosticos.append({
                "fecha": fecha_str,
                "ciudad": ciudad,
                "temperatura": item["main"]["temp"],
                "humedad": item["main"]["humidity"],
                "presion": item["main"]["pressure"],
                "tipo": "pronostico"
            })
    return pronosticos

def guardar_csv(nombre_archivo, datos):
    campos = ["fecha", "ciudad", "temperatura", "humedad", "presion", "tipo"]
    try:
        with open(nombre_archivo, mode='w', newline='', encoding='utf-8') as archivo:
            writer = csv.DictWriter(archivo, fieldnames=campos)
            writer.writeheader()
            writer.writerows(datos)
        print(f"✅ Datos guardados en {nombre_archivo}")
    except Exception as e:
        print(f"❌ Error al guardar CSV: {e}")

def guardar_json(nombre_archivo, datos):
    try:
        with open(nombre_archivo, mode='w', encoding='utf-8') as archivo:
            json.dump(datos, archivo, indent=4)
        print(f"✅ Datos guardados en {nombre_archivo}")
    except Exception as e:
        print(f"❌ Error al guardar JSON: {e}")

def main():
    ciudad = input("🏙️ Introduce el nombre de la ciudad para obtener el clima: ").strip()
    if not ciudad:
        print("❌ Debes ingresar un nombre de ciudad válido.")
        return

    clima_actual = obtener_clima_actual(ciudad)
    if clima_actual is None:
        return

    pronostico = obtener_pronostico(ciudad)
    if not pronostico:
        print("⚠️ No se pudo obtener pronóstico, solo se guardará el clima actual.")
        datos_guardar = [clima_actual]
    else:
        datos_guardar = [clima_actual] + pronostico

    # Guardar en archivos fijos sin fecha ni hora en el nombre
    guardar_csv("clima_datos.csv", datos_guardar)
    guardar_json("clima_datos.json", datos_guardar)

    print("\n📊 Clima actual y pronóstico (solo temperaturas y humedad):")
    for dato in datos_guardar:
        tipo = "Actual" if dato["tipo"] == "actual" else "Pronóstico"
        print(f"{tipo} - {dato['fecha']} - {dato['ciudad']}: {dato['temperatura']}°C, Humedad: {dato['humedad']}%, Presión: {dato['presion']} hPa")

if __name__ == "__main__":
    main()


**Script de análisis:**

In [ ]:
import csv
import json
import statistics

def leer_datos_csv(nombre_archivo):
    datos = []
    try:
        with open(nombre_archivo, mode='r', encoding='utf-8') as archivo:
            lector = csv.DictReader(archivo)
            for fila in lector:
                datos.append({
                    "fecha": fila["fecha"],
                    "ciudad": fila["ciudad"],
                    "temperatura": float(fila["temperatura"]),
                    "humedad": float(fila["humedad"]),
                    "presion": float(fila["presion"]),
                    "tipo": fila["tipo"]
                })
    except FileNotFoundError:
        print(f"❌ Archivo CSV no encontrado: {nombre_archivo}")
    except Exception as e:
        print(f"❌ Error leyendo CSV: {e}")
    return datos

def leer_datos_json(nombre_archivo):
    datos = []
    try:
        with open(nombre_archivo, mode='r', encoding='utf-8') as archivo:
            json_data = json.load(archivo)
            for item in json_data:
                datos.append({
                    "fecha": item["fecha"],
                    "ciudad": item["ciudad"],
                    "temperatura": float(item["temperatura"]),
                    "humedad": float(item["humedad"]),
                    "presion": float(item["presion"]),
                    "tipo": item["tipo"]
                })
    except FileNotFoundError:
        print(f"❌ Archivo JSON no encontrado: {nombre_archivo}")
    except Exception as e:
        print(f"❌ Error leyendo JSON: {e}")
    return datos

def analizar_datos(datos):
    if not datos:
        print("⚠️ No hay datos para analizar.")
        return

    temperaturas = [d["temperatura"] for d in datos]
    humedades = [d["humedad"] for d in datos]
    presiones = [d["presion"] for d in datos]

    print("\n📈 Resultados del análisis:")

    print(f"🌡️ Temperatura promedio: {statistics.mean(temperaturas):.2f} °C")
    if len(temperaturas) > 1:
        print(f"📉 Desviación estándar de la temperatura: {statistics.stdev(temperaturas):.2f} °C")
    else:
        print("⚠️ No hay suficientes datos para calcular la desviación estándar de la temperatura.")

    print(f"💧 Humedad promedio: {statistics.mean(humedades):.2f} %")

    print(f"🧭 Rango de presión atmosférica: {min(presiones):.2f} - {max(presiones):.2f} hPa")

    ciudades = set(d["ciudad"] for d in datos)
    fechas = set(d["fecha"] for d in datos)
    print(f"🏙️ Ciudades analizadas: {', '.join(ciudades)}")
    print(f"📅 Días analizados: {', '.join(fechas)}")

if __name__ == "__main__":
    archivo_csv = "clima_datos.csv"
    archivo_json = "clima_datos.json"

    # Cambia aquí a True para probar el archivo CSV o JSON
    usar_csv = True

    if usar_csv:
        print(f"📄 Analizando archivo CSV: {archivo_csv}")
        datos = leer_datos_csv(archivo_csv)
    else:
        print(f"📄 Analizando archivo JSON: {archivo_json}")
        datos = leer_datos_json(archivo_json)

    analizar_datos(datos)


📄 Analizando archivo CSV: clima_datos.csv

📈 Resultados del análisis:
🌡️ Temperatura promedio: 36.94 °C
📉 Desviación estándar de la temperatura: 2.62 °C
💧 Humedad promedio: 27.00 %
🧭 Rango de presión atmosférica: 1003.00 - 1007.00 hPa
🏙️ Ciudades analizadas: Apodaca
📅 Días analizados: 2025-05-20, 2025-05-18, 2025-05-17, 2025-05-19, 2025-05-16, 2025-05-15


# **Código principal y análisis**

Habiendo terminado ambos scripts, podemos juntarlos en un único código que primero muestre el clima de hoy junto con el pronóstico a 5 días, y luego realice un análisis en base a los datos guardados en csv o json.

In [ ]:
import requests
import csv
import json
from datetime import datetime
import statistics

API_KEY = "f0f0c07c117721896152aeca0fbbe31a"

def get_weather_current(city):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    res = requests.get(url)
    if res.status_code != 200:
        print(f"❌ Error: Ciudad '{city}' no encontrada o API falló.")
        return None
    data = res.json()
    return {
        "fecha": datetime.utcfromtimestamp(data["dt"]).strftime("%Y-%m-%d"),
        "ciudad": data["name"],
        "temperatura": data["main"]["temp"],
        "humedad": data["main"]["humidity"],
        "presion": data["main"]["pressure"],
        "tipo": "actual"
    }

def get_weather_forecast(city):
    url = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
    res = requests.get(url)
    if res.status_code != 200:
        print(f"❌ Error: Ciudad '{city}' no encontrada o API falló.")
        return []
    data = res.json()
    forecast_data = []
    # Filtrar solo pronóstico de las 18:00 cada día
    for item in data["list"]:
        dt_txt = item["dt_txt"]  # formato "YYYY-MM-DD HH:MM:SS"
        if dt_txt.endswith("18:00:00"):
            forecast_data.append({
                "fecha": dt_txt.split()[0],
                "ciudad": data["city"]["name"],
                "temperatura": item["main"]["temp"],
                "humedad": item["main"]["humidity"],
                "presion": item["main"]["pressure"],
                "tipo": "pronostico"
            })
    return forecast_data

def guardar_csv(datos, nombre_archivo):
    campos = ["fecha", "ciudad", "temperatura", "humedad", "presion", "tipo"]
    with open(nombre_archivo, mode='w', encoding='utf-8', newline='') as archivo:
        escritor = csv.DictWriter(archivo, fieldnames=campos)
        escritor.writeheader()
        escritor.writerows(datos)

def guardar_json(datos, nombre_archivo):
    with open(nombre_archivo, mode='w', encoding='utf-8') as archivo:
        json.dump(datos, archivo, indent=4, ensure_ascii=False)

def leer_datos_csv(nombre_archivo):
    datos = []
    try:
        with open(nombre_archivo, mode='r', encoding='utf-8') as archivo:
            lector = csv.DictReader(archivo)
            for fila in lector:
                datos.append({
                    "fecha": fila["fecha"],
                    "ciudad": fila["ciudad"],
                    "temperatura": float(fila["temperatura"]),
                    "humedad": float(fila["humedad"]),
                    "presion": float(fila["presion"]),
                    "tipo": fila["tipo"]
                })
    except FileNotFoundError:
        print(f"❌ Archivo CSV no encontrado: {nombre_archivo}")
    except Exception as e:
        print(f"❌ Error leyendo CSV: {e}")
    return datos

def leer_datos_json(nombre_archivo):
    datos = []
    try:
        with open(nombre_archivo, mode='r', encoding='utf-8') as archivo:
            json_data = json.load(archivo)
            for item in json_data:
                datos.append({
                    "fecha": item["fecha"],
                    "ciudad": item["ciudad"],
                    "temperatura": float(item["temperatura"]),
                    "humedad": float(item["humedad"]),
                    "presion": float(item["presion"]),
                    "tipo": item["tipo"]
                })
    except FileNotFoundError:
        print(f"❌ Archivo JSON no encontrado: {nombre_archivo}")
    except Exception as e:
        print(f"❌ Error leyendo JSON: {e}")
    return datos

def analizar_datos(datos):
    if not datos:
        print("⚠️ No hay datos para analizar.")
        return

    temperaturas = [d["temperatura"] for d in datos]
    humedades = [d["humedad"] for d in datos]
    presiones = [d["presion"] for d in datos]

    print("\n📈 Resultados del análisis:")

    print(f"🌡️ Temperatura promedio: {statistics.mean(temperaturas):.2f} °C")
    if len(temperaturas) > 1:
        print(f"📉 Desviación estándar de la temperatura: {statistics.stdev(temperaturas):.2f} °C")
    else:
        print("⚠️ No hay suficientes datos para calcular la desviación estándar de la temperatura.")

    print(f"💧 Humedad promedio: {statistics.mean(humedades):.2f} %")

    print(f"🧭 Rango de presión atmosférica: {min(presiones):.2f} - {max(presiones):.2f} hPa")

    ciudades = set(d["ciudad"] for d in datos)
    fechas = set(d["fecha"] for d in datos)
    print(f"🏙️ Ciudades analizadas: {', '.join(ciudades)}")
    print(f"📅 Días analizados: {', '.join(fechas)}")

def mostrar_datos(actual, pronostico):
    print(f"\n🌆 Clima actual en {actual['ciudad']} ({actual['fecha']}):")
    print(f"  Temperatura: {actual['temperatura']} °C")
    print(f"  Humedad: {actual['humedad']} %")
    print(f"  Presión: {actual['presion']} hPa")

    print("\n📅 Pronóstico a 5 días (18:00 hrs):")
    for p in pronostico:
        print(f"  {p['fecha']} -> Temp: {p['temperatura']} °C, Humedad: {p['humedad']} %, Presión: {p['presion']} hPa")

def main():
    ciudad = input("🌆 Ingrese la ciudad para consultar el clima: ").strip()
    if not ciudad:
        print("❌ No ingresó ninguna ciudad.")
        return

    actual = get_weather_current(ciudad)
    if actual is None:
        return

    pronostico = get_weather_forecast(ciudad)

    mostrar_datos(actual, pronostico)

    # Guardar solo el clima actual y pronóstico
    datos_guardar = [actual] + pronostico

    guardar_csv(datos_guardar, "clima_datos.csv")
    guardar_json(datos_guardar, "clima_datos.json")

    print(f"\n✅ Datos guardados en 'clima_datos.csv' y 'clima_datos.json'")

    # Leer datos para análisis
    datos_csv = leer_datos_csv("clima_datos.csv")

    print("\n--- Análisis basado en CSV ---")
    analizar_datos(datos_csv)

if __name__ == "__main__":
    main()


🌆 Ingrese la ciudad para consultar el clima: Apodaca

🌆 Clima actual en Apodaca (2025-05-15):
  Temperatura: 39.88 °C
  Humedad: 14 %
  Presión: 1004 hPa

📅 Pronóstico a 5 días (18:00 hrs):
  2025-05-16 -> Temp: 35.65 °C, Humedad: 32 %, Presión: 1006 hPa
  2025-05-17 -> Temp: 35.11 °C, Humedad: 38 %, Presión: 1007 hPa
  2025-05-18 -> Temp: 34.14 °C, Humedad: 39 %, Presión: 1007 hPa
  2025-05-19 -> Temp: 36.57 °C, Humedad: 29 %, Presión: 1004 hPa
  2025-05-20 -> Temp: 39.17 °C, Humedad: 10 %, Presión: 1006 hPa

✅ Datos guardados en 'clima_datos.csv' y 'clima_datos.json'

--- Análisis basado en CSV ---

📈 Resultados del análisis:
🌡️ Temperatura promedio: 36.75 °C
📉 Desviación estándar de la temperatura: 2.30 °C
💧 Humedad promedio: 27.00 %
🧭 Rango de presión atmosférica: 1004.00 - 1007.00 hPa
🏙️ Ciudades analizadas: Apodaca
📅 Días analizados: 2025-05-20, 2025-05-18, 2025-05-17, 2025-05-19, 2025-05-16, 2025-05-15


# **Documentación del Proyecto: Análisis de Clima Actual y Pronóstico a 5 Días**
**1. Objetivo del Proyecto**

El proyecto consiste en consultar, mostrar y analizar datos meteorológicos (clima actual y pronóstico a 5 días) para una ciudad dada, utilizando la API pública de OpenWeather. El análisis ayuda a entender la temperatura promedio, humedad y presión atmosférica, para evaluar la estabilidad climática y posibles variaciones.

**2. Proceso de Extracción y Lectura de Datos**

Obtención de Datos:
API utilizada: OpenWeather (https://openweathermap.org/api)

**Datos solicitados:**

Clima actual (/weather endpoint)

Pronóstico a 5 días cada 3 horas (/forecast endpoint)

**Parámetros:**

Ciudad ingresada por el usuario.

Unidades métricas (grados Celsius).

API Key para autenticación.

**Lectura y Validación:**
Se consultan los endpoints de la API con requests.

Se verifica el código de respuesta HTTP (200 OK).

Si la respuesta es inválida o la ciudad no existe, se notifica y el programa termina.

De los datos JSON recibidos, se extraen solo campos esenciales:

Fecha

Ciudad

Temperatura (°C)

Humedad (%)

Presión atmosférica (hPa)

Tipo (actual o pronóstico)

Validación en el Código:
Comprobación de existencia y estado de respuesta.

Conversión segura a tipos numéricos (float) al leer archivos CSV o JSON para evitar errores en cálculos.

Manejo de excepciones para archivos no encontrados o datos malformados.

**3. Preparación y Almacenamiento de Datos**

Se guarda la información actual y el pronóstico (filtrado solo para las 18:00 hrs de cada día) en dos formatos:

CSV (clima_datos.csv)

JSON (clima_datos.json)

El nombre de archivo es fijo para facilitar la lectura posterior y evitar problemas con fechas o nombres variables.

Solo se guardan datos relevantes para análisis posteriores, no se almacenan datos auxiliares de la API.

**4. Análisis de Datos**

**Lectura para Análisis:**
Los datos se leen desde los archivos CSV o JSON.

Se parsean y convierten a estructuras de Python (listas de diccionarios).

Se filtran y convierten los valores numéricos para cálculos estadísticos.

**Cálculos Estadísticos:**

**Temperatura promedio:** Media aritmética de todas las temperaturas recogidas.

Desviación estándar de temperatura: Indica la variabilidad de las temperaturas.

**Humedad promedio:** Media de los valores de humedad.

**Rango de presión atmosférica:** Diferencia entre la presión mínima y máxima observadas, para detectar variaciones.

**Resultados ejemplo:**

In [ ]:
🌡️ Temperatura promedio: 21.85 °C
📉 Desviación estándar de la temperatura: 2.34 °C
💧 Humedad promedio: 65.20 %
🧭 Rango de presión atmosférica: 1010.00 - 1022.00 hPa
🏙️ Ciudades analizadas: CiudadEjemplo
📅 Días analizados: 2025-05-15, 2025-05-16, 2025-05-17

Estos valores ayudan a responder preguntas como:

¿Cuál es la temperatura promedio durante los días analizados?

¿Qué tan estable o variable es la temperatura (desviación estándar)?

¿Cómo varía la humedad relativa?

¿Existen cambios notables en la presión atmosférica que puedan indicar cambios meteorológicos?

**5. Relación con el Problema Definido**
El problema inicial era obtener y analizar información meteorológica para evaluar la estabilidad y variabilidad climática en una ciudad específica. El código desarrollado permite:

Consultar datos en tiempo real y el pronóstico a 5 días.

Guardar solo los datos esenciales para evitar almacenamiento innecesario.

Calcular estadísticas básicas que permiten evaluar la estabilidad del clima.

Por ejemplo, una baja desviación estándar en la temperatura y un rango estrecho en la presión indicarían un clima estable durante el período analizado, mientras que valores más altos pueden sugerir cambios abruptos o inestabilidad.

**6. Consideraciones y Mejoras Futuras**

Se podrían agregar más variables para análisis, como velocidad del viento o precipitación.

Implementar análisis visual con gráficos para facilitar la interpretación.

Automatizar la ejecución diaria para obtener datos en diferentes horas.

Incluir validaciones más avanzadas y manejo de errores.
